# Speech Understanding
## Assignment 1
#### Question 1
### Akansha Gautam (M23CSA506)

# Importing Libraries

In [38]:
import numpy as np
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
import IPython.display as ipd
import librosa.display
import torchaudio
import torchaudio.transforms as transforms
import requests
import csv
import jiwer

pd.set_option('display.max_colwidth', None)

# Set Device

In [2]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"Using device: {device}")

Using device: mps


## Load the LJ Speech Dataset

In [22]:
# https://www.kaggle.com/datasets/mathurinache/the-lj-speech-dataset

column_names = ["file_name", "transcription", "normalized_transcription"]
metadata = pd.read_csv('/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/metadata.csv', 
                       delimiter='|', names=column_names)
metadata['file_path'] = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/' + metadata.file_name.astype(str) + '.wav'

metadata.head()

,file_name,transcription,normalized_transcription,file_path
0,LJ001-0001,"Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition","Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition",/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/LJ001-0001.wav
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/LJ001-0002.wav
2,LJ001-0003,"For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process","For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process",/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/LJ001-0003.wav
3,LJ001-0004,"produced the block books, which were the immediate predecessors of the true printed book,","produced the block books, which were the immediate predecessors of the true printed book,",/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/LJ001-0004.wav
4,LJ001-0005,the invention of movable metal letters in the middle of the fifteenth century may justly be considered as the invention of the art of printing.,the invention of movable metal letters in the middle of the fifteenth century may justly be considered as the invention of the art of printing.,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/LJSpeech-1.1/wavs/LJ001-0005.wav


In [23]:
API_KEY = "4109d1c8defd962ff4bc6dda9e1ca0b1cb66e9e4"
MAX_METADATA_FILES = 1000

In [26]:
url = "https://api.deepgram.com/v1/listen?model=nova-2&smart_format=true"

headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "audio/wav"
}

nova_2_transcripts = []

for i in range(MAX_METADATA_FILES):
    file_path = metadata.iloc[i]['file_path']
    with open(file_path, "rb") as audio_file:
        response = requests.post(url, headers=headers, data=audio_file).json()
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
    nova_2_transcripts.append(transcript)
    print(f"iteration i: {i} | transcript: {transcript}")
    print() 


iteration i: 0 | transcript: Printing, in the only sense with which we are at present concerned, differs from most, if not from all, the arts and crafts represented in the exhibition,

iteration i: 1 | transcript: Being comparatively modern,

iteration i: 2 | transcript: For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands by a similar process,

iteration i: 3 | transcript: Produced the block books, which were the immediate predecessors of the true printed book,

iteration i: 4 | transcript: The invention of movable metal letters in the middle of 15th century may justly be considered as the invention of the art of printing.

iteration i: 5 | transcript: And, it is worth mentioning in passing, that as an example of fine typography,

iteration i: 6 | transcript: The earliest book printed with movable types, the Gutenberg, or forty two line Bible, of about 1455.

iteration i: 7 | transcript: Has never been su

In [31]:
def write_list_to_csv(data_list, file_name):
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        for item in data_list:
            writer.writerow([item])

write_list_to_csv(nova_2_transcripts, 'nova-2-transcripts.csv')

In [36]:
nova_transcripts = pd.read_csv('/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 1/nova-2-transcripts.csv', names=['generated_transcription'])
nova_transcripts.head(3)

,generated_transcription
0,"Printing, in the only sense with which we are at present concerned, differs from most, if not from all, the arts and crafts represented in the exhibition,"
1,"Being comparatively modern,"
2,"For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands by a similar process,"


In [45]:
def calculate_wer(original_transcriptions, generated_transcriptions):
    wer_scores = []
    
    for i in range(len(generated_transcriptions)):
        wer = jiwer.wer(original_transcriptions[i], generated_transcriptions[i])
        wer_scores.append(wer)
    
    return sum(wer_scores) / len(wer_scores) if wer_scores else 0

wer_result = calculate_wer(metadata["transcription"], nova_transcripts['generated_transcription'])
print(f"Average WER: {wer_result:.2%}")

Average WER: 15.48%
